# Python API Example - Access DES Hub Netbacks 

__N.B. This guide is just for Access DES Hub Netbacks. If you're looking for other API data products (such as Contract prices, Freight routes or Netbacks), please refer to their according code example files.__ 

### Have any questions?

If you have any questions regarding our API, or need help accessing specific datasets, please contact us at:

__data@sparkcommodities.com__


## 1. Importing Data

Here we define the functions that allow us to retrieve the valid credentials to access the Spark API.

This section can remain unchanged for most Spark API users.

In [123]:
# import libraries for callin the API
import json
import os
import sys
import pandas as pd
from base64 import b64encode
from urllib.parse import urljoin
from pprint import pprint

try:
    from urllib import request, parse
    from urllib.error import HTTPError
except ImportError:
    raise RuntimeError("Python 3 required")

In [124]:
# defining query functions 
API_BASE_URL = "https://api.sparkcommodities.com"


def retrieve_credentials(file_path=None):
    """
    Find credentials either by reading the client_credentials file or reading
    environment variables
    """
    if file_path is None:
        client_id = os.getenv("SPARK_CLIENT_ID")
        client_secret = os.getenv("SPARK_CLIENT_SECRET")
        if not client_id or not client_secret:
            raise RuntimeError(
                "SPARK_CLIENT_ID and SPARK_CLIENT_SECRET environment vars required"
            )
    else:
        # Parse the file
        if not os.path.isfile(file_path):
            raise RuntimeError("The file {} doesn't exist".format(file_path))

        with open(file_path) as fp:
            lines = [l.replace("\n", "") for l in fp.readlines()]

        if lines[0] in ("clientId,clientSecret", "client_id,client_secret"):
            client_id, client_secret = lines[1].split(",")
        else:
            print("First line read: '{}'".format(lines[0]))
            raise RuntimeError(
                "The specified file {} doesn't look like to be a Spark API client "
                "credentials file".format(file_path)
            )

    print(">>>> Found credentials!")
    print(
        ">>>> Client_id={}****, client_secret={}****".format(
            client_id[:5], client_secret[:5]
        )
    )

    return client_id, client_secret


def do_api_post_query(uri, body, headers):
    """
    OAuth2 authentication requires a POST request with client credentials before accessing the API. 
    This POST request will return an Access Token which will be used for the API GET request.
    """
    url = urljoin(API_BASE_URL, uri)

    data = json.dumps(body).encode("utf-8")

    # HTTP POST request
    req = request.Request(url, data=data, headers=headers)
    try:
        response = request.urlopen(req)
    except HTTPError as e:
        print("HTTP Error: ", e.code)
        print(e.read())
        sys.exit(1)

    resp_content = response.read()

    # The server must return HTTP 201. Raise an error if this is not the case
    assert response.status == 201, resp_content

    # The server returned a JSON response
    content = json.loads(resp_content)

    return content


def do_api_get_query(uri, access_token):
    """
    After receiving an Access Token, we can request information from the API.
    """
    url = urljoin(API_BASE_URL, uri)

    headers = {
        "Authorization": "Bearer {}".format(access_token),
        "accept": "application/json",
    }

    print(f"Fetching {url}")

    # HTTP GET request
    req = request.Request(url, headers=headers)
    try:
        response = request.urlopen(req)
    except HTTPError as e:
        print("HTTP Error: ", e.code)
        print(e.read())
        sys.exit(1)

    resp_content = response.read()

    # The server must return HTTP 201. Raise an error if this is not the case
    assert response.status == 200, resp_content

    # The server returned a JSON response
    content = json.loads(resp_content)

    return content


def get_access_token(client_id, client_secret):
    """
    Get a new access_token. Access tokens are the thing that applications use to make
    API requests. Access tokens must be kept confidential in storage.

    # Procedure:

    Do a POST query with `grantType` and `scopes` in the body. A basic authorization
    HTTP header is required. The "Basic" HTTP authentication scheme is defined in
    RFC 7617, which transmits credentials as `clientId:clientSecret` pairs, encoded
    using base64.
    """

    # Note: for the sake of this example, we choose to use the Python urllib from the
    # standard lib. One should consider using https://requests.readthedocs.io/

    payload = "{}:{}".format(client_id, client_secret).encode()
    headers = {
        "Authorization": b64encode(payload).decode(),
        "Accept": "application/json",
        "Content-Type": "application/json",
    }
    body = {
        "grantType": "clientCredentials",
        "scopes": "read:access",
    }

    content = do_api_post_query(uri="/oauth/token/", body=body, headers=headers)

    print(
        ">>>> Successfully fetched an access token {}****, valid {} seconds.".format(
            content["accessToken"][:5], content["expiresIn"]
        )
    )

    return content["accessToken"]

## N.B. Credentials

Here we call the above functions, and input the file path to our credentials.

N.B. You must have downloaded your client credentials CSV file before proceeding. Please refer to the API documentation if you have not dowloaded them already.

The code then prints the available prices that are callable from the API, and their corresponding Python ticker names are displayed as a list at the bottom of the Output.

In [ ]:
# Insert file path to your client credentials here
client_id, client_secret = retrieve_credentials(file_path="/tmp/client_credentials.csv")

# Authenticate:
access_token = get_access_token(client_id, client_secret)

>>>> Found credentials!
>>>> Client_id=01c23****, client_secret=80763****
>>>> Successfully fetched an access token eyJhb****, valid 604799 seconds.


## 2. Importing Data
Here we define the function to call the DES Hub Netbacks endpoint. The only required parameter is "unit", which can be set as "usd-per-mmbtu" or "eur-per-mwh". For full details on the input parameters, please refer to our API Website:

https://www.sparkcommodities.com/api/lng-access/des-hub-netbacks.html

In [126]:
## Defining the function to import the data

def fetch_price_releases(access_token, unit, limit=None, offset=None, terminal=None):
    
    query_params = "?unit={}".format(unit)
    if limit is not None:
        query_params += "&limit={}".format(limit)
    if offset is not None:
        query_params += "&offset={}".format(offset)
    if terminal is not None:
        query_params += "&terminal={}".format(terminal)


    content = do_api_get_query(
        uri="/beta/access/des-hub-netbacks/{}".format(query_params), access_token=access_token
    )

    return content


## Calling that function and storing the output
historical = fetch_price_releases(access_token, unit='usd-per-mmbtu')

Fetching https://api.sparkcommodities.com/beta/access/des-hub-netbacks/?unit=usd-per-mmbtu


In [127]:
# Showing the raw JSON output
historical

{'errors': [],
 'data': [{'releaseDate': '2025-05-12',
   'deliveryMonth': '2025-06-01',
   'monthIndex': 'M+1',
   'terminalUuid': '0038a35c-253f-44f5-a4e5-d5240d98039a',
   'netbackTtfBasis': '-0.608',
   'netbackOutright': '10.917',
   'totalRegasificationCost': '1.401',
   'slotUnloadStorageRegas': '1.098',
   'slotBerth': '0.057',
   'slotBerthUnloadStorageRegas': '0.0',
   'additionalStorage': '0.0',
   'additionalSendout': '0.031',
   'gasInKind': '0.093',
   'entryCapacity': '0.122',
   'commodityCharge': '0.0',
   'otherCosts': None},
  {'releaseDate': '2025-05-12',
   'deliveryMonth': '2025-07-01',
   'monthIndex': 'M+2',
   'terminalUuid': '0038a35c-253f-44f5-a4e5-d5240d98039a',
   'netbackTtfBasis': '-0.535',
   'netbackOutright': '11.06',
   'totalRegasificationCost': '1.402',
   'slotUnloadStorageRegas': '1.098',
   'slotBerth': '0.057',
   'slotBerthUnloadStorageRegas': '0.0',
   'additionalStorage': '0.0',
   'additionalSendout': '0.031',
   'gasInKind': '0.094',
   'en

In [128]:
# Sorting the JSON into a Pandas DataFrame

def organise_dataframe(data):
    """
    This function sorts the API content into a dataframe. The columns available are Release Date, Terminal, Month, Vessel Size, $/MMBtu and €/MWh. 
    Essentially, this function parses the Access database using the Month, Terminal and Vessel size columns as reference.
    """
    # create columns
    data_dict = {
        'Release Date':[],
        'Terminal':[],
        'Month Index':[],
        'Delivery Month':[],
        'DES Hub Netback - TTF Basis':[],
        'DES Hub Netback - Outright':[],
        'Total Regas':[],
        'Basic Slot (Berth)':[],
        'Basic Slot (Unload/Stor/Regas)':[],
        'Basic Slot (B/U/S/R)':[],
        'Additional Storage':[],
        'Additional Sendout':[],
        'Gas in Kind': [],
        'Entry Capacity':[],
        'Commodity Charge':[]
    }

    # loop for each Terminal
    for l in data['data']:
        
        # assigning values to each column
        data_dict['Release Date'].append(l["releaseDate"])
        data_dict['Terminal'].append(data['metaData']['terminals'][l['terminalUuid']])
        data_dict['Month Index'].append(l['monthIndex'])
        data_dict['Delivery Month'].append(l['deliveryMonth'])

        data_dict['DES Hub Netback - TTF Basis'].append(float(l['netbackTtfBasis']))
        data_dict['DES Hub Netback - Outright'].append(float(l['netbackOutright']))
        data_dict['Total Regas'].append(float(l['totalRegasificationCost']))
        data_dict['Basic Slot (Berth)'].append(float(l['slotBerth']))
        data_dict['Basic Slot (Unload/Stor/Regas)'].append(float(l['slotUnloadStorageRegas']))
        data_dict['Basic Slot (B/U/S/R)'].append(float(l['slotBerthUnloadStorageRegas']))
        data_dict['Additional Storage'].append(float(l['additionalStorage']))
        data_dict['Additional Sendout'].append(float(l['additionalSendout']))
        data_dict['Gas in Kind'].append(float(l['gasInKind']))
        data_dict['Entry Capacity'].append(float(l['entryCapacity']))
        data_dict['Commodity Charge'].append(float(l['commodityCharge']))
                
    
    # convert into dataframe
    df = pd.DataFrame(data_dict)
    
    df['Delivery Month'] = pd.to_datetime(df['Delivery Month'])
    df['Release Date'] = pd.to_datetime(df['Release Date'])
    
    return df


### Storing as a DataFrame
We can use the 'organise_dataframe' function to parse the content into a dataframe

In [129]:
# Storing the "historical" JSON as a Pandas Dataframe using the organise_dataframe function
hist_df = organise_dataframe(historical)
hist_df.head(3)

,Release Date,Terminal,Month Index,Delivery Month,DES Hub Netback - TTF Basis,DES Hub Netback - Outright,Total Regas,Basic Slot (Berth),Basic Slot (Unload/Stor/Regas),Basic Slot (B/U/S/R),Additional Storage,Additional Sendout,Gas in Kind,Entry Capacity,Commodity Charge
0,2025-05-12,adriatic,M+1,2025-06-01,-0.608,10.917,1.401,0.057,1.098,0.0,0.0,0.031,0.093,0.122,0.0
1,2025-05-12,adriatic,M+2,2025-07-01,-0.535,11.060,1.402,0.057,1.098,0.0,0.0,0.031,0.094,0.122,0.0
2,2025-05-12,adriatic,M+3,2025-08-01,-0.681,10.997,1.402,0.057,1.098,0.0,0.0,0.031,0.094,0.122,0.0


## Calling Data for a specific terminal

The DES Hub Netbacks endpoint also provides the option to call data for a specific terminal. This is done using the "terminal" optional parameter - input any terminal name into this parameter and the endpoint will retrieve the relevant data.

For a list of available terminal names, refer to the metadata section of the JSON output called above. Example shown below, with the left column showing Terminal UUIDs, and the right column showing Terminal codes:

In [130]:
# List of available terminal names
historical['metaData']['terminals']

{'0038a35c-253f-44f5-a4e5-d5240d98039a': 'adriatic',
 '003497c6-ed32-412f-95ef-c3b1f962464e': 'brunsbuttel',
 '003e3e70-3626-4124-8ee9-d3ec39678e8c': 'deutsche-ostsee',
 '00338f3f-8875-435d-87a9-f83d9a5c5241': 'dunkerque',
 '0037d9e4-cf09-4f26-8934-f1e038e185ea': 'eems-energy-terminal',
 '003b1adb-f810-443c-a971-c2a6b28cb5dc': 'fos-cavaou',
 '003f577c-7058-4b50-9c94-c499c07ca080': 'gate',
 '00317185-978a-4df5-970c-2c28d3ab893c': 'grain-lng',
 '00361ab8-f70d-4a08-8e45-e6eb5a0b8b2f': 'le-havre',
 '003b1d25-f4bd-43bf-9cf6-9bd38216fe0f': 'montoir',
 '00355021-dc45-4aaa-8178-a6dc360c07b9': 'olt-toscana',
 '003660ee-567d-4d23-9e43-2891509b7bfb': 'piombino',
 '0030d930-6574-4049-a739-327a16620429': 'ravenna',
 '003bf9ab-2829-40a1-a83d-c32b764f21fd': 'south-hook',
 '003b319e-b29e-4853-b4ee-85794d5bacba': 'stade',
 '0032d353-d0d8-4454-9b6c-a6c5db12e49d': 'tvb',
 '003b1d36-d72b-4331-888f-22b3f84c1cce': 'wilhelmshaven',
 '00378624-8afa-4cce-987e-e76cebe077ab': 'wilhelmshaven-2',
 '0031994e-f370-4

In [131]:
# calling historical data for 1 specific terminal (in this case Dunkerque)

historical_term = fetch_price_releases(access_token, unit='usd-per-mmbtu', terminal='dunkerque')
term_df = organise_dataframe(historical_term)
term_df.head(3)

Fetching https://api.sparkcommodities.com/beta/access/des-hub-netbacks/?unit=usd-per-mmbtu&terminal=dunkerque


,Release Date,Terminal,Month Index,Delivery Month,DES Hub Netback - TTF Basis,DES Hub Netback - Outright,Total Regas,Basic Slot (Berth),Basic Slot (Unload/Stor/Regas),Basic Slot (B/U/S/R),Additional Storage,Additional Sendout,Gas in Kind,Entry Capacity,Commodity Charge
0,2025-05-12,dunkerque,M+1,2025-06-01,-1.099,10.426,0.725,0.0,0.0,0.622,0.0,0.0,0.0,0.103,0.0
1,2025-05-12,dunkerque,M+2,2025-07-01,-1.069,10.526,0.725,0.0,0.0,0.622,0.0,0.0,0.0,0.103,0.0
2,2025-05-12,dunkerque,M+3,2025-08-01,-1.073,10.605,0.725,0.0,0.0,0.622,0.0,0.0,0.0,0.103,0.0


## N.B. Historical Data Limits

Currently, a maximum of 30 historical datasets can be called at one time due to the size of the data file. 

If more data points are required, the below code can be used. It calls 30 historical datasets at a time, but utilises the 'offset' parameter to call datasets further back in the historical database. To call more history, increase the 'n_offset' parameter in the first line of the code. The 'n_offset' parameter describes the number of historical data requests to be executed.

In [132]:
def loop_historical_data(token,n_offset):
    # initalise first set of historical data and initialising dataframe
    historical = fetch_price_releases(access_token, unit='usd-per-mmbtu', limit=30)
    hist_df = organise_dataframe(historical)

    # Looping through earlier historical data and adding to the historical dataframe
    for i in range(1,n_offset+1):
        historical = fetch_price_releases(access_token, unit='usd-per-mmbtu', limit=30, offset=i*30)
        hist_df = pd.concat([hist_df,organise_dataframe(historical)])

    return hist_df

In [133]:
hdf = loop_historical_data(access_token,2)
hdf.head(3)

Fetching https://api.sparkcommodities.com/beta/access/des-hub-netbacks/?unit=usd-per-mmbtu&limit=30
Fetching https://api.sparkcommodities.com/beta/access/des-hub-netbacks/?unit=usd-per-mmbtu&limit=30&offset=30
Fetching https://api.sparkcommodities.com/beta/access/des-hub-netbacks/?unit=usd-per-mmbtu&limit=30&offset=60


,Release Date,Terminal,Month Index,Delivery Month,DES Hub Netback - TTF Basis,DES Hub Netback - Outright,Total Regas,Basic Slot (Berth),Basic Slot (Unload/Stor/Regas),Basic Slot (B/U/S/R),Additional Storage,Additional Sendout,Gas in Kind,Entry Capacity,Commodity Charge
0,2025-05-12,adriatic,M+1,2025-06-01,-0.608,10.917,1.401,0.057,1.098,0.0,0.0,0.031,0.093,0.122,0.0
1,2025-05-12,adriatic,M+2,2025-07-01,-0.535,11.060,1.402,0.057,1.098,0.0,0.0,0.031,0.094,0.122,0.0
2,2025-05-12,adriatic,M+3,2025-08-01,-0.681,10.997,1.402,0.057,1.098,0.0,0.0,0.031,0.094,0.122,0.0
